In [1]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
from pymc.math import concatenate
import aesara.tensor as at

In [2]:
yellow = np.array([45, 59, 48, 46, 38, 47])
white = np.array([21, 12, 14, 17, 13, 17])
green = np.array([37, 32, 15, 25, 39, 41])
blue = np.array([16, 11, 20, 21, 14, 7])
bugs = np.stack([yellow, white, green, blue], axis=0)
bugs

array([[45, 59, 48, 46, 38, 47],
       [21, 12, 14, 17, 13, 17],
       [37, 32, 15, 25, 39, 41],
       [16, 11, 20, 21, 14,  7]])

In [75]:
with pm.Model() as m_shapes:
    mu0 = pm.Normal("mu0", mu=0, tau=.0001)
    tau = pm.Gamma("tau", .001, .001)
    
    alpha2_4 = pm.Normal("alpha2_4", mu=0, tau=.0001, shape=(3, 6))
    alpha1 = pm.Deterministic("alpha1", at.reshape(-pm.math.sum(alpha2_4, axis=0), (1, 6), ndim=2))
    alpha = pm.Deterministic("alpha", pm.math.concatenate([alpha1, alpha2_4], axis=0))

    mu = mu0 + alpha

    lik = pm.Normal("lik", mu=mu, tau=tau, observed=bugs)
    
    trace = pm.sample(5000)
    
az.summary(trace.posterior.alpha.mean(axis=(3)), kind="stats")

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu0, tau, alpha2_4]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 14 seconds.
The acceptance probability does not match the target. It is 0.8851, but should be close to 0.8. Try to increase the

In [77]:
az.summary(trace.posterior.alpha.mean(axis=(3)), kind="stats")

,mean,sd,hdi_3%,hdi_97%
alpha[0],19.871,1.788,16.514,23.283
alpha[1],-11.611,1.811,-14.945,-8.249
alpha[2],4.181,1.796,0.870,7.523
alpha[3],-12.441,1.773,-15.757,-9.140


In [82]:
with pm.Model() as m_shapes2:
    mu0 = pm.Normal("mu0", mu=0, tau=.0001)
    tau = pm.Gamma("tau", .001, .001)
    
    alpha2_4 = pm.Normal("alpha2_4", mu=0, tau=.0001, shape=(3, 6))
    alpha1 = pm.Deterministic("alpha1", -pm.math.sum(alpha2_4, axis=0))
    alpha = pm.Deterministic("alpha", pm.math.concatenate([alpha1, alpha2_4], axis=0))

    mu = mu0 + alpha

    lik = pm.Normal("lik", mu=mu, tau=tau, observed=bugs)
    
    trace = pm.sample(5000)

TypeError: Only tensors with the same number of dimensions can be joined

In [16]:
az.summary(trace, hdi_prob=.95, var_names=["~alpha2_4"])

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu0,27.285,1.026,25.280,29.333,0.006,0.004,33775.0,10233.0,1.0
tau,0.066,0.041,0.005,0.146,0.001,0.001,2447.0,4083.0,1.0
"alpha1[0, 0]",15.286,4.305,6.321,23.805,0.026,0.023,26841.0,10024.0,1.0
"alpha1[0, 1]",30.510,4.205,21.449,38.450,0.026,0.019,25112.0,10853.0,1.0
"alpha1[0, 2]",23.745,4.248,15.239,32.120,0.026,0.020,25255.0,11040.0,1.0
"alpha1[0, 3]",18.779,4.262,10.283,27.417,0.028,0.022,23922.0,10253.0,1.0
"alpha1[0, 4]",11.965,4.285,3.296,20.542,0.026,0.022,26780.0,10898.0,1.0
"alpha1[0, 5]",19.023,4.373,10.577,27.774,0.028,0.022,24935.0,10603.0,1.0
"alpha[0, 0]",15.286,4.305,6.321,23.805,0.026,0.023,26841.0,10024.0,1.0
"alpha[0, 1]",30.510,4.205,21.449,38.450,0.026,0.019,25112.0,10853.0,1.0


In [21]:
with pm.Model() as m_easy:
    mu0 = pm.Normal("mu0", mu=0, tau=.0001)
    tau = pm.Gamma("tau", .001, .001)

    alpha4 = pm.Normal("alpha4", mu=0, tau=.0001)
    alpha3 = pm.Normal("alpha3", mu=0, tau=.0001)
    alpha2 = pm.Normal("alpha2", mu=0, tau=.0001)
    alpha1 = pm.Deterministic("alpha1", -(alpha2 + alpha3 + alpha4))
    
    mu_1 = mu0 + alpha1
    mu_2 = mu0 + alpha2
    mu_3 = mu0 + alpha3
    mu_4 = mu0 + alpha4

    lik1 = pm.Normal("lik1", mu=mu_1, tau=tau, observed=yellow)
    lik2 = pm.Normal("lik2", mu=mu_2, tau=tau, observed=white)
    lik3 = pm.Normal("lik3", mu=mu_3, tau=tau, observed=green)
    lik4 = pm.Normal("lik4", mu=mu_4, tau=tau, observed=blue)
    
    onetwo = pm.Deterministic("onetwo", alpha1-alpha2)
    onethree = pm.Deterministic("onethree", alpha1-alpha3)
    onefour = pm.Deterministic("onefour", alpha1-alpha4)
    twothree = pm.Deterministic("twothree", alpha2-alpha3)
    twofour = pm.Deterministic("twofour", alpha2-alpha4)
    threefour = pm.Deterministic("threefour", alpha3-alpha4)

    
    trace = pm.sample(5000)

az.summary(trace, hdi_prob=.95)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu0, tau, alpha4, alpha3, alpha2]


/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc-dev-py39/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 12 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu0,27.303,1.434,24.421,30.080,0.011,0.008,17838.0,12750.0,1.0
alpha4,-12.468,2.483,-17.486,-7.699,0.019,0.013,17593.0,14724.0,1.0
alpha3,4.222,2.492,-0.749,9.098,0.019,0.015,16801.0,14177.0,1.0
alpha2,-11.596,2.513,-16.643,-6.690,0.021,0.015,14993.0,12663.0,1.0
tau,0.022,0.007,0.009,0.036,0.000,0.000,12258.0,10318.0,1.0
alpha1,19.842,2.518,14.749,24.700,0.016,0.012,23974.0,15440.0,1.0
onetwo,31.437,4.107,23.147,39.366,0.030,0.021,18875.0,15110.0,1.0
onethree,15.619,4.102,7.536,23.770,0.028,0.020,20770.0,16691.0,1.0
onefour,32.310,4.086,24.289,40.385,0.028,0.020,21840.0,17169.0,1.0
twothree,-15.818,4.092,-23.808,-7.746,0.034,0.024,14932.0,11617.0,1.0
